In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from tqdm import tqdm
import joblib
import glob
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical


IMG_HEIGHT = 128
IMG_WIDTH = 128
RAW_DIR = "../data/raw"
PROCESSED_DIR = "../data/processed"
AUGMENTED_DIR = "../data/augmented"
MODEL_DIR = "../models"
NUM_CLASSES = 2  # healthy / diseased

os.makedirs(PROCESSED_DIR, exist_ok=True)
os.makedirs(AUGMENTED_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)


In [2]:
df = pd.read_csv(os.path.join(RAW_DIR, "train.csv"))
print(" Исходные данные:", df.shape)
df.head()

df["label"] = df.apply(lambda row: 0 if row["healthy"] == 1 else 1, axis=1)
df = df[["image_id", "label"]]

 Исходные данные: (1821, 5)


In [3]:
def load_images(df, folder, size=(128, 128)):
    images_flat = []
    images_cnn = []
    labels = []
    for _, row in tqdm(df.iterrows(), total=len(df)):
        img_path = os.path.join(folder, row["image_id"] + ".jpg")
        if not os.path.exists(img_path):
            continue
        img = cv2.imread(img_path)
        img = cv2.resize(img, size)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        images_flat.append(img_rgb.flatten())

        images_cnn.append(img_rgb)
        
        labels.append(row["label"])
        
    return (np.array(images_flat), 
            np.array(images_cnn, dtype=np.float32) / 255.0,  # нормализация для CNN
            np.array(labels))

X_flat, X_cnn, y = load_images(df, os.path.join(RAW_DIR, "images"))\

print("Загружено изображений (flatten):", X_flat.shape)
print("Загружено изображений (CNN):", X_cnn.shape)


  0%|                                                                                         | 0/1821 [00:00<?, ?it/s]

  0%|▎                                                                                | 6/1821 [00:00<00:30, 58.98it/s]

  1%|▌                                                                               | 14/1821 [00:00<00:27, 66.33it/s]

  1%|▉                                                                               | 21/1821 [00:00<00:27, 65.34it/s]

  2%|█▏                                                                              | 28/1821 [00:00<00:27, 64.04it/s]

...

 99%|█████████████████████████████████████████████████████████████████████████████▏| 1801/1821 [00:28<00:00, 62.78it/s]

 99%|█████████████████████████████████████████████████████████████████████████████▍| 1808/1821 [00:29<00:00, 60.98it/s]

100%|█████████████████████████████████████████████████████████████████████████████▋| 1815/1821 [00:29<00:00, 61.72it/s]

100%|██████████████████████████████████████████████████████████████████████████████| 1821/1821 [00:29<00:00, 62.23it/s]

Загружено изображений (flatten): (1821, 49152)
Загружено изображений (CNN): (1821, 128, 128, 3)


In [4]:
y_cnn = to_categorical(y, NUM_CLASSES)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_flat, y, test_size=0.2, random_state=42)
X_train_cnn, X_test_cnn, y_train_cnn, y_test_cnn = train_test_split(X_cnn, y_cnn, test_size=0.2, random_state=42)



In [6]:
np.save(os.path.join(PROCESSED_DIR, "X_flat.npy"), X_train)
np.save(os.path.join(PROCESSED_DIR, "X_test_flat.npy"), X_test)
np.save(os.path.join(PROCESSED_DIR, "X_cnn.npy"), X_train_cnn)
np.save(os.path.join(PROCESSED_DIR, "X_test_cnn.npy"), X_test_cnn)
np.save(os.path.join(PROCESSED_DIR, "y.npy"), y_train)
np.save(os.path.join(PROCESSED_DIR, "y_test.npy"), y_test)
np.save(os.path.join(PROCESSED_DIR, "y_cnn.npy"), y_train_cnn)
np.save(os.path.join(PROCESSED_DIR, "y_test_cnn.npy"), y_test_cnn)


In [7]:
def create_cnn_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
        MaxPooling2D(2,2),
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [8]:
models = {
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=42),
    "LogisticRegression": LogisticRegression(max_iter=200),
    "CNN": "custom"
}


results = {}

In [9]:

os.makedirs(MODEL_DIR, exist_ok=True)

results = {}

for name, model in models.items():
    print(f"\nОбучение модели: {name}")

    if name != "CNN":
        model.fit(X_train, y_train)
        preds = model.predict(X_test)
        acc = accuracy_score(y_test, preds)
        results[name] = acc
        print(f"Accuracy {name}: {acc:.4f}")
        joblib.dump(model, os.path.join(MODEL_DIR, f"{name}.joblib"))

    else:
        print("Создание и обучение CNN...")
        cnn_model = create_cnn_model(
            input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
            num_classes=NUM_CLASSES
        )

        history = cnn_model.fit(
            X_train_cnn, y_train_cnn,
            validation_data=(X_test_cnn, y_test_cnn),
            epochs=10,
            batch_size=32,
            verbose=1
        )

        loss, acc = cnn_model.evaluate(X_test_cnn, y_test_cnn, verbose=0)
        results["CNN"] = acc
        print(f"Accuracy CNN: {acc:.4f}")

        cnn_model.save(os.path.join(MODEL_DIR, "CNN.h5"))



Обучение модели: RandomForest


Accuracy RandomForest: 0.7616

Обучение модели: LogisticRegression


Accuracy LogisticRegression: 0.6603

Обучение модели: CNN
Создание и обучение CNN...
Epoch 1/10


C:\Users\sb170\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


 1/46 ━━━━━━━━━━━━━━━━━━━━ 33s 744ms/step - accuracy: 0.6875 - loss: 0.6684

...

43/46 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.9294 - loss: 0.1705

44/46 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.9294 - loss: 0.1705

45/46 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.9294 - loss: 0.1706

46/46 ━━━━━━━━━━━━━━━━━━━━ 4s 80ms/step - accuracy: 0.9295 - loss: 0.1707 - val_accuracy: 0.7562 - val_loss: 0.6866


Accuracy CNN: 0.7562


In [10]:
metrics = pd.DataFrame(list(results.items()), columns=["Model", "Accuracy"])
metrics.to_csv("metrics.csv", index=False)
metrics

,Model,Accuracy
0,RandomForest,0.761644
1,LogisticRegression,0.660274
2,CNN,0.756164


In [11]:
aug_gen = ImageDataGenerator(
    rotation_range=25,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode="nearest"
)

In [12]:

img_folder = os.path.join(RAW_DIR, "images")
save_folder = os.path.join(AUGMENTED_DIR, "images")
os.makedirs(save_folder, exist_ok=True)

aug_images = []
aug_labels = []

In [13]:
for idx, row in tqdm(df.iterrows(), total=len(df)):
    img_path = os.path.join(img_folder, row["image_id"] + ".jpg")
    if not os.path.exists(img_path):
        continue
    img = cv2.imread(img_path)
    img = cv2.resize(img, (IMG_HEIGHT, IMG_WIDTH))
    img = img.reshape((1,) + img.shape)
    
    count = 0
    for batch in aug_gen.flow(img, batch_size=1, save_to_dir=save_folder,
                              save_prefix=row["image_id"], save_format='jpg'):
        aug_images.append(batch[0].astype(np.float32)/255.0)  # для CNN
        aug_labels.append(row["label"])
        count += 1
        if count >= 2:
            break

print("Всего аугментированных изображений:", len(aug_images))

  0%|                                                                                         | 0/1821 [00:00<?, ?it/s]

  0%|▏                                                                                | 4/1821 [00:00<00:46, 38.98it/s]

  0%|▍                                                                                | 9/1821 [00:00<00:41, 43.94it/s]

  1%|▋                                                                               | 15/1821 [00:00<00:38, 46.63it/s]

  1%|▉                                                                               | 20/1821 [00:00<00:39, 45.89it/s]

...

 99%|█████████████████████████████████████████████████████████████████████████████▌| 1810/1821 [00:39<00:00, 43.88it/s]

100%|█████████████████████████████████████████████████████████████████████████████▋| 1815/1821 [00:39<00:00, 43.23it/s]

100%|█████████████████████████████████████████████████████████████████████████████▉| 1820/1821 [00:39<00:00, 42.52it/s]

100%|██████████████████████████████████████████████████████████████████████████████| 1821/1821 [00:39<00:00, 45.81it/s]

Всего аугментированных изображений: 3642


In [14]:
from sklearn.preprocessing import StandardScaler
X_aug_flat = np.array([img.flatten() for img in aug_images])
y_aug_flat = np.array(aug_labels)

X_all_flat = np.concatenate([X_flat, X_aug_flat])
y_all_flat = np.concatenate([y, y_aug_flat])

X_train2_flat, X_test2_flat, y_train2_flat, y_test2_flat = train_test_split(
    X_all_flat, y_all_flat, test_size=0.2, random_state=42
)
scaler = StandardScaler()
X_train2_flat_scaled = scaler.fit_transform(X_train2_flat)
X_test2_flat_scaled = scaler.transform(X_test2_flat)
X_aug_cnn = np.array(aug_images)
y_aug_cnn = tf.keras.utils.to_categorical(aug_labels, num_classes=NUM_CLASSES)

X_train2_cnn = np.concatenate([X_train_cnn, X_aug_cnn])
y_train2_cnn = np.concatenate([y_train_cnn, y_aug_cnn])

X_test2_cnn = X_test_cnn
y_test2_cnn = y_test_cnn

In [15]:
results_aug = {}

for name, model in models.items():
    print(f"\nОбучение модели (аугм. данные): {name}")
    
    if name != "CNN":
        model.fit(X_train2_flat, y_train2_flat)
        preds = model.predict(X_test2_flat)
        acc = accuracy_score(y_test2_flat, preds)
        results_aug[name] = acc
    else:
        cnn_model = create_cnn_model((IMG_HEIGHT, IMG_WIDTH, 3), num_classes=NUM_CLASSES)
        cnn_model.fit(
            X_train2_cnn, y_train2_cnn,
            validation_data=(X_test2_cnn, y_test2_cnn),
            epochs=10,
            batch_size=32,
            verbose=1
        )
        loss, acc = cnn_model.evaluate(X_test2_cnn, y_test2_cnn, verbose=0)
        results_aug["CNN"] = acc
        cnn_model.save(os.path.join(MODEL_DIR, "CNN_aug.h5"))
    
    print(f"Accuracy {name} (aug): {results_aug[name]:.4f}")


Обучение модели (аугм. данные): RandomForest


Accuracy RandomForest (aug): 0.7640

Обучение модели (аугм. данные): LogisticRegression


C:\Users\sb170\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy LogisticRegression (aug): 0.6880

Обучение модели (аугм. данные): CNN


C:\Users\sb170\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

157/160 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.8811 - loss: 0.2672

158/160 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.8811 - loss: 0.2672

159/160 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.8812 - loss: 0.2672

160/160 ━━━━━━━━━━━━━━━━━━━━ 13s 83ms/step - accuracy: 0.8812 - loss: 0.2672 - val_accuracy: 0.7534 - val_loss: 0.7753


Accuracy CNN (aug): 0.7534


In [16]:
metrics_aug = pd.DataFrame(list(results_aug.items()), columns=["Model", "Accuracy (Augmented)"])
metrics_aug.to_csv("metrics_aug.csv", index=False)

merged = pd.merge(metrics, metrics_aug, on="Model")
print("\nСравнение точностей:")


Сравнение точностей:
